In [2]:
import numpy as np

epsilon = 0.001
epochs = 1

In [2]:
np.minimum([0,1,1,0],[0,0,1,1])

array([0, 0, 1, 0])

In [2]:
def load_data(filename):
    X_data = []
    y_data = []
    with open(filename) as file:
        for line in file:
            splittedLine = line.split(" ")
            X = splittedLine[:256]
            X = np.array([0 if float(i) == 0. else 1 for i in X])
            X_data.append(X)

            y = splittedLine[256:-1]
            y = [int(i) for i in y]
            y = y.index(1)
            y_data.append(y)

    return np.array(X_data), np.array(y_data)

In [3]:
def shuffle(a, b):
    pos = np.random.permutation(len(a))
    return a[pos], b[pos]

In [8]:
def addComplement(X):
    X_data = []
    for x in X:
        complement = np.array([i ^ 1 for i in x])
        X_data.append(np.concatenate((x, complement), axis=None))

    return np.array(X_data)


In [9]:
def calculate_T(I, w, alpha):
    T = np.sum(np.minimum(I, w)) / (alpha + np.sum(w))
    return T

In [10]:
def addNeuron(layer, n):
    w = np.ones(n)
    layer.append([w, 0, -1])  # weights,committed/uncommitted, class label
    return layer


In [34]:
def run(X, y, alpha, beta, vigilance):
    second_layer = []

    for epoch in range(epochs):
        for it in range(X.shape[0]):
            if len(second_layer) == 0:
                second_layer = addNeuron(second_layer, X.shape[1])

            T = []
            step = 1
            J = -1
            l_vigilance = -1
            while True:
                if step == 1:
                    print('1----set the vigilance factor equal to its baseline value')
                    l_vigilance = vigilance
                    step = 2
                    
               
                if step == 2:
                    print('2---insert input and calculate second layers activities')
                    for i in range(len(second_layer)):
                        if second_layer[i][1] == 0:
                            t = np.random.random()
                            print("if",t)
                        else:
                            t = calculate_T(X[it], second_layer[i][0], alpha)
                            print("else", t)
                        T.append(t)

                    step = 3
                
               
                if step == 3: 
                    print('3----find the winner neuron')
                    J = np.argmax(T)
                    
                    print(J)
                    print('3.2----check if the neuron is commited')
                    if second_layer[J][1] == 0: 
                        step = 7
                    else:
                        step = 4
                
                if step == 4:
                    print('4----check vigilance to see if the input is similar enough to the winner s prototype')
                    if np.sum(np.minimum(X[it], second_layer[J][0])) / np.sum(X[it]) >= l_vigilance:
                        step = 5
                    else: 
                        print('4else-----check next winner')
                        T[J] = -1
                        step = 3
                
                
                if step == 5:
                    print('5----check if winner class label matches with the input class label')
                    if second_layer[J][2] == y[it]:
                        w_new = beta * np.minimum(X[it], second_layer[J][0]) + (1 - beta) * second_layer[J][0]
                        second_layer[J][0] = w_new
                        step = 9
                    else: 
                        T[J] = -1
                        l_vigilance = np.sum(np.minimum(X[it], second_layer[J][0])) / np.sum(X[it]) + epsilon
                        step = 6
                
                if step == 6:
                    print('6---#check if vigilance to terminate the training in the current epoch ')
                    if l_vigilance > 1:
                        step = 9
                    else:
                        step = 3
               
                if step == 7:
                    print('7--create new subclass')
                    second_layer[J][0] = X[it]
                    second_layer[J][1] = 1
                    second_layer[J][2] = y[it]
                    step = 8
                
                
                if step == 8:
                    print('8--create a new uncommitted neuron')
                    second_layer = addNeuron(second_layer, X.shape[1])
                    step = 9
                
               
                if step == 9:
                    print()
                    print(second_layer)
                    print()
                    print(T)
                    print('neeeext')
                    print()
                    break

    return second_layer

In [35]:
X=np.array([[0,0],[0,1],[1,0],[1,1]])
y=np.array([0,1,1,0])
alpha=beta=1
vigilance=0.5
X=addComplement(X)
aux=run(X, y, alpha, beta, vigilance)

1----set the vigilance factor equal to its baseline value
2---insert input and calculate second layers activities
if 0.8721342190296142
3----find the winner neuron
0
3.2----check if the neuron is commited
7--create new subclass
8--create a new uncommitted neuron

[[array([0, 0, 1, 1]), 1, 0], [array([1., 1., 1., 1.]), 0, -1]]

[0.8721342190296142]
neeeext

1----set the vigilance factor equal to its baseline value
2---insert input and calculate second layers activities
else 0.3333333333333333
if 0.929184598576625
3----find the winner neuron
1
3.2----check if the neuron is commited
7--create new subclass
8--create a new uncommitted neuron

[[array([0, 0, 1, 1]), 1, 0], [array([0, 1, 1, 0]), 1, 1], [array([1., 1., 1., 1.]), 0, -1]]

[0.3333333333333333, 0.929184598576625]
neeeext

1----set the vigilance factor equal to its baseline value
2---insert input and calculate second layers activities
else 0.3333333333333333
else 0.0
if 0.9116673628183717
3----find the winner neuron
2
3.2----check

In [15]:
aux

[[array([0, 0, 1, 1]), 1, 0],
 [array([0, 1, 1, 0]), 1, 1],
 [array([1, 0, 0, 1]), 1, 1],
 [array([1, 1, 0, 0]), 1, 0],
 [array([1., 1., 1., 1.]), 0, -1]]

In [36]:
def test(second_layer, X, y, alpha):
    nr = 0
    for it in range(X.shape[0]):
        T = []
        for i in range(len(second_layer)):
            t = calculate_T(X[it], second_layer[i][0], alpha)
            T.append(t)

        J = np.argmax(T)
        if second_layer[J][2] == y[it]:
            nr += 1

    return 100 * nr / X.shape[0]

In [37]:
test(aux,X,y,alpha)

100.0

In [10]:
X, y = load_data("semeion.data");
X, y = shuffle(X, y)
X = addComplement(X)

In [11]:
nr_train = int(70 * X.shape[0] / 100)
X_train = X[:nr_train, :]
y_train = y[:nr_train]
X_test = X[nr_train:, :]
y_test = y[nr_train:]

In [12]:
nr_train_train = int(70 * X_train.shape[0] / 100)
X_train_train = X_train[:nr_train_train, :]
X_train_valid = X_train[nr_train_train:, :]
y_train_train = y_train[:nr_train_train]
y_train_valid = y_train[nr_train_train:]

In [13]:
alpha = [0.01, 0.1, 0.2, 0.3, 0.4]
beta = [1, 0.8, 0.6, 0.4]
vigilance = [0.01, 0.1, 0.2, 0.3]

In [14]:
best_alpha = -1
best_beta = -1
best_vigilance = -1
maximum_accuracy = -1

In [15]:
for al in alpha:
    for be in beta:
        for vi in vigilance:
            second_layer = run(X_train_train, y_train_train, al, be, vi)
            acc = test(second_layer[:-1], X_train_valid, y_train_valid, al)
            print(f"Param: alpha= {al}, beta= {be}, vigilance= {vi}")
            print(f"Accuracy train validation= {acc}")
            if acc > maximum_accuracy:
                best_alpha = al
                best_beta = be
                best_vigilance = vi
                maximum_accuracy = acc

second_layer = run(X_train, y_train, best_alpha, best_beta, best_vigilance)
acc = test(second_layer[:-1], X_test, y_test, best_alpha)
print()
print(f"Param: alpha= {best_alpha}, beta= {best_beta}, vigilance= {best_vigilance}")
print(f"Acuracy test= {acc}")

Param: alpha= 0.01, beta= 1, vigilance= 0.01
Accuracy train validation= 75.22388059701493
Param: alpha= 0.01, beta= 1, vigilance= 0.1
Accuracy train validation= 78.2089552238806
Param: alpha= 0.01, beta= 1, vigilance= 0.2
Accuracy train validation= 80.8955223880597
Param: alpha= 0.01, beta= 1, vigilance= 0.3
Accuracy train validation= 88.95522388059702
Param: alpha= 0.01, beta= 0.8, vigilance= 0.01
Accuracy train validation= 73.13432835820896
Param: alpha= 0.01, beta= 0.8, vigilance= 0.1
Accuracy train validation= 76.41791044776119
Param: alpha= 0.01, beta= 0.8, vigilance= 0.2
Accuracy train validation= 87.16417910447761
Param: alpha= 0.01, beta= 0.8, vigilance= 0.3
Accuracy train validation= 87.16417910447761
Param: alpha= 0.01, beta= 0.6, vigilance= 0.01
Accuracy train validation= 75.22388059701493
Param: alpha= 0.01, beta= 0.6, vigilance= 0.1
Accuracy train validation= 80.59701492537313
Param: alpha= 0.01, beta= 0.6, vigilance= 0.2
Accuracy train validation= 82.98507462686567
Param: